# 📊 03 - Unión de Dataset (Dinámico)

**TFM: Predicción de Abandono Universitario**

Este notebook:
- Une todas las tablas en df_alumno
- Genera reporte Sweetviz de df_alumno
- Actualiza transformaciones_dinamico.html con df_alumno

**Autora:** María José Morte (morte@uji.es)

## 1. Configuración Inicial

In [1]:
# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar rutas
import sys
sys.path.append('../src')

try:
    from config import DATA_INTERIM, DATA_PROCESSED, DOCS, info_entorno
    info_entorno()
except ImportError:
    DATA_INTERIM = Path('../data/02_interim')
    DATA_PROCESSED = Path('../data/03_processed')
    DOCS = Path('../docs')
    print(f"Usando rutas por defecto")

# Crear carpetas si no existen
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
DOCS.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Configuración cargada")

INFORMACIÓN DEL ENTORNO
Entorno: local
Raíz proyecto: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_
Data RAW: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\01_raw
Data INTERIM: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\02_interim
Data PROCESSED: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\03_processed
Docs: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\docs

✅ Configuración cargada


## 2. Cargar Tablas

In [2]:
# =============================================================================
# CARGAR TODAS LAS TABLAS
# =============================================================================

print("="*60)
print("CARGANDO TABLAS")
print("="*60)

df_exp = pd.read_parquet(DATA_INTERIM / 'expedientes_limpio.parquet')
print(f"  Expedientes: {len(df_exp):,} registros")

df_tit = pd.read_parquet(DATA_INTERIM / 'titulaciones_limpio.parquet')
print(f"  Titulaciones: {len(df_tit):,} registros")

df_nac = pd.read_parquet(DATA_INTERIM / 'nac_sexo_limpio.parquet')
print(f"  Nac_sexo: {len(df_nac):,} registros")

df_dom = pd.read_parquet(DATA_INTERIM / 'domicilios_limpio.parquet')
print(f"  Domicilios: {len(df_dom):,} registros")

df_becas = pd.read_parquet(DATA_INTERIM / 'becas_limpio.parquet')
print(f"  Becas: {len(df_becas):,} registros")

df_trabajo = pd.read_parquet(DATA_INTERIM / 'trabajo_limpio.parquet')
print(f"  Trabajo: {len(df_trabajo):,} registros")

df_notas = pd.read_parquet(DATA_INTERIM / 'notas_limpio.parquet')
print(f"  Notas: {len(df_notas):,} registros")

df_recibos = pd.read_parquet(DATA_INTERIM / 'recibos_limpio.parquet')
print(f"  Recibos: {len(df_recibos):,} registros")

CARGANDO TABLAS
  Expedientes: 109,575 registros
  Titulaciones: 45 registros
  Nac_sexo: 30,873 registros
  Domicilios: 109,206 registros
  Becas: 70,524 registros
  Trabajo: 195,524 registros
  Notas: 107,908 registros
  Recibos: 114,447 registros


## 3. Crear df_alumno

In [3]:
# =============================================================================
# CREAR df_alumno - UNIR TODAS LAS TABLAS
# =============================================================================

print("\n" + "="*60)
print("CREANDO df_alumno")
print("="*60)

# Base: Expedientes
print("\n1. Base: Expedientes...")
df_alumno = df_exp.copy()
print(f"   Registros: {len(df_alumno):,}")

# + Titulaciones
print("\n2. Añadiendo Titulaciones...")
df_alumno = df_alumno.merge(
    df_tit[['exp_tit_id', 'titulacion_nombre', 'rama', 'cred_titulacion']],
    on='exp_tit_id',
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Nac_sexo
print("\n3. Añadiendo Nac_sexo...")
df_alumno = df_alumno.merge(
    df_nac,
    on='per_id_ficticio',
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Domicilios
print("\n4. Añadiendo Domicilios...")
df_alumno = df_alumno.merge(
    df_dom,
    on=['per_id_ficticio', 'curso_aca'],
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Becas (puede haber varias, cogemos la primera)
print("\n5. Añadiendo Becas...")
df_becas_unico = df_becas.drop_duplicates(subset=['per_id_ficticio', 'curso_aca'], keep='first')
df_alumno = df_alumno.merge(
    df_becas_unico,
    on=['per_id_ficticio', 'curso_aca'],
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Trabajo
print("\n6. Añadiendo Trabajo...")
df_trabajo_unico = df_trabajo.drop_duplicates(subset=['per_id_ficticio', 'curso_aca'], keep='first')
df_alumno = df_alumno.merge(
    df_trabajo_unico,
    on=['per_id_ficticio', 'curso_aca'],
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Notas
print("\n7. Añadiendo Notas...")
df_notas_unico = df_notas.drop_duplicates(subset=['per_id_ficticio', 'curso_aca'], keep='first')
df_alumno = df_alumno.merge(
    df_notas_unico,
    on=['per_id_ficticio', 'curso_aca'],
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")

# + Recibos
print("\n8. Añadiendo Recibos...")
df_recibos_unico = df_recibos.drop_duplicates(subset=['per_id_ficticio', 'curso_aca'], keep='first')
df_alumno = df_alumno.merge(
    df_recibos_unico,
    on=['per_id_ficticio', 'curso_aca'],
    how='left'
)
print(f"   Registros: {len(df_alumno):,}")


CREANDO df_alumno

1. Base: Expedientes...
   Registros: 109,575

2. Añadiendo Titulaciones...
   Registros: 109,575

3. Añadiendo Nac_sexo...
   Registros: 109,575

4. Añadiendo Domicilios...
   Registros: 109,575

5. Añadiendo Becas...
   Registros: 109,575

6. Añadiendo Trabajo...
   Registros: 109,575

7. Añadiendo Notas...
   Registros: 109,575

8. Añadiendo Recibos...
   Registros: 109,575


## 4. Resumen df_alumno

In [4]:
# =============================================================================
# RESUMEN df_alumno
# =============================================================================

print("\n" + "="*60)
print("RESUMEN df_alumno")
print("="*60)
print(f"Registros: {len(df_alumno):,}")
print(f"Columnas: {len(df_alumno.columns)}")
print(f"\nColumnas y % nulos:")
for col in df_alumno.columns:
    nulos = df_alumno[col].isna().sum()
    pct = nulos / len(df_alumno) * 100
    print(f"   {col}: {pct:.1f}%")


RESUMEN df_alumno
Registros: 109,575
Columnas: 30

Columnas y % nulos:
   per_id_ficticio: 0.0%
   exp_tit_id: 0.0%
   curso_aca: 0.0%
   curso_aca_fin: 51.1%
   nota_1: 52.5%
   via_acceso: 0.0%
   seguro: 0.0%
   nota_selectividad_exp: 45.0%
   nota_acceso_exp: 10.0%
   cred_matriculados: 0.0%
   cred_superados: 0.0%
   egresado: 0.0%
   nuevo: 0.0%
   media_curso: 7.2%
   titulacion_nombre: 0.0%
   rama: 0.0%
   cred_titulacion: 0.0%
   sexo: 0.0%
   pais: 0.0%
   edad: 0.0%
   poblacion: 0.0%
   provincia: 0.0%
   pais_domicilio: 0.0%
   tipo_domicilio: 0.0%
   beca: 46.9%
   trabajo: 45.9%
   media_titulacion_curso: 4.1%
   media_alumno_curso: 4.1%
   forma_pago: 0.0%
   numero_pagos: 0.0%


## 5. Guardar df_alumno

In [5]:
# =============================================================================
# GUARDAR df_alumno
# =============================================================================

print("\n" + "="*60)
print("GUARDANDO df_alumno")
print("="*60)

# Parquet
path_parquet = DATA_PROCESSED / 'df_alumno.parquet'
df_alumno.to_parquet(path_parquet, index=False)
print(f"✅ Guardado: {path_parquet.name}")

# CSV (para Power BI / Tableau)
path_csv = DATA_PROCESSED / 'df_alumno.csv'
df_alumno.to_csv(path_csv, index=False, encoding='utf-8-sig')
print(f"✅ Guardado: {path_csv.name}")


GUARDANDO df_alumno
✅ Guardado: df_alumno.parquet
✅ Guardado: df_alumno.csv


## 6. Generar Reporte Sweetviz de df_alumno

In [6]:
# =============================================================================
# GENERAR REPORTE SWEETVIZ DE df_alumno
# =============================================================================

print("\n" + "="*60)
print("GENERANDO REPORTE SWEETVIZ DE df_alumno")
print("="*60)

# Parchear numpy
import numpy as np
if not hasattr(np, 'VisibleDeprecationWarning'):
    np.VisibleDeprecationWarning = np.exceptions.VisibleDeprecationWarning

import sweetviz as sv

print("Procesando... (puede tardar 1-2 minutos)")
reporte = sv.analyze(df_alumno, pairwise_analysis='off')

# Guardar
html_path = DOCS / 'reporte_df_alumno.html'
reporte.show_html(str(html_path), open_browser=False)

print(f"✅ Guardado: {html_path.name}")


GENERANDO REPORTE SWEETVIZ DE df_alumno
Procesando... (puede tardar 1-2 minutos)


                                             |          | [  0%]   00:00 -> (? left)

Report C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\docs\reporte_df_alumno.html was generated.
✅ Guardado: reporte_df_alumno.html


## 7. Actualizar transformaciones_dinamico.html

In [7]:
# =============================================================================
# ACTUALIZAR transformaciones_dinamico.html CON df_alumno
# =============================================================================

print("\n" + "="*60)
print("ACTUALIZANDO transformaciones_dinamico.html")
print("="*60)

trans_path = DOCS / 'transformaciones_dinamico.html'

# Leer HTML existente
with open(trans_path, 'r', encoding='utf-8') as f:
    html = f.read()

# Reemplazar el div placeholder de df_alumno por el enlace real
old_div = '''<div id="df-alumno-container" class="bg-gray-300 rounded-xl p-6 text-gray-600 text-center">
                    <div class="text-2xl font-bold">🎯 df_alumno.parquet</div>
                    <div class="text-lg opacity-90">Pendiente de generar</div>
                    <div class="text-sm opacity-75 mt-2">Ejecuta 03_union_dataset_dinamico.ipynb</div>
                </div>'''

new_div = f'''<a href="reporte_df_alumno.html" target="_blank" class="df-alumno-link transition-all">
                    <div class="bg-gradient-to-r from-green-400 to-blue-500 rounded-xl p-6 text-white text-center shadow-lg hover:shadow-2xl cursor-pointer">
                        <div class="text-2xl font-bold">🎯 df_alumno.parquet</div>
                        <div class="text-lg opacity-90">Dataset unificado para modelado</div>
                        <div class="text-sm opacity-75 mt-2">{len(df_alumno):,} registros | {len(df_alumno.columns)} columnas</div>
                        <div class="text-xs mt-3 bg-white/20 rounded px-3 py-1 inline-block">📊 Clic para ver Reporte Sweetviz</div>
                    </div>
                </a>'''

html = html.replace(old_div, new_div)

# Guardar
with open(trans_path, 'w', encoding='utf-8') as f:
    f.write(html)

print(f"✅ Actualizado: {trans_path.name}")
print(f"   df_alumno: {len(df_alumno):,} registros, {len(df_alumno.columns)} columnas")


ACTUALIZANDO transformaciones_dinamico.html
✅ Actualizado: transformaciones_dinamico.html
   df_alumno: 109,575 registros, 30 columnas


## 8. Resumen Final

In [8]:
# =============================================================================
# RESUMEN FINAL
# =============================================================================

print("\n" + "="*60)
print("RESUMEN FINAL - FASE 1 COMPLETADA")
print("="*60)

print("\n📁 FICHEROS GENERADOS:")

print("\ndata/03_processed/:")
for f in sorted(DATA_PROCESSED.glob('*')):
    if f.is_file() and not f.name.startswith('.'):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

print("\ndocs/:")
for f in sorted(DOCS.glob('*.html')):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name} ({size_kb:.1f} KB)")

print("\n" + "="*60)
print("✅ FASE 1 COMPLETADA")
print("="*60)
print("\nPara ver los resultados:")
print("  Abre docs/transformaciones_dinamico.html en tu navegador")
print("\nSiguiente paso: FASE 2 - EDA (Análisis Exploratorio)")


RESUMEN FINAL - FASE 1 COMPLETADA

📁 FICHEROS GENERADOS:

data/03_processed/:
  df_alumno.csv (26728.4 KB)
  df_alumno.parquet (1927.1 KB)

docs/:
  reporte_becas.html (591.1 KB)
  reporte_df_alumno.html (1856.6 KB)
  reporte_domicilios.html (683.0 KB)
  reporte_expedientes.html (1185.9 KB)
  reporte_nac_sexo.html (592.1 KB)
  reporte_notas.html (676.4 KB)
  reporte_preinscripcion.html (906.1 KB)
  reporte_recibos.html (568.7 KB)
  reporte_titulaciones.html (580.0 KB)
  reporte_trabajo.html (585.9 KB)
  transformaciones_dinamico.html (29.9 KB)

✅ FASE 1 COMPLETADA

Para ver los resultados:
  Abre docs/transformaciones_dinamico.html en tu navegador

Siguiente paso: FASE 2 - EDA (Análisis Exploratorio)
